In [12]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import urllib
from pathlib import Path
from typing import List
import pandas as pd
import numpy as np
import re
import nltk

COMMENTS_FILE : Path =  Path(".") / "comments.tsv"
ANNOTATIONS_FILE : Path = Path(".") / "annotations.tsv"

ANNOTATED_COMMENTS_URL : str = 'https://ndownloader.figshare.com/files/7554634' 
ANNOTATIONS_URL : str = 'https://ndownloader.figshare.com/files/7554637' 


def download_dataset(url: str, f: Path):
    """
    download_dataset: download dataset from the given URL
    """
    if not f.exists():
        urllib.request.urlretrieve(url, f)

# Download the Dataset
download_dataset(ANNOTATED_COMMENTS_URL, COMMENTS_FILE)
download_dataset(ANNOTATIONS_URL, ANNOTATIONS_FILE)

# load data into dataset
annotations : pd.DataFrame = pd.read_csv(ANNOTATIONS_FILE, sep = "\t")
comments : pd.DataFrame = pd.read_csv(COMMENTS_FILE, sep = "\t")
comments = comments.set_index("rev_id")

In [13]:
# compute the labels of the data
attack_prob : pd.DataFrame = annotations.groupby("rev_id")["attack"].mean()
# Do some data cleaning => (NaNs => 0, )
comments["attack"] = attack_prob
comments = comments.fillna(0)
comments["comment"] = comments["comment"].str.replace(
    re.compile("TAB_TOKEN|NEWLINE_TOKEN"), " ")
comments["comment"] = comments["comment"].apply(lambda x:  nltk.word_tokenize(x))


In [14]:
comments.head()

,comment,year,logged_in,ns,sample,split,attack
rev_id,,,,,,,
37675,"[`, -, This, is, not, ``, creative, ``, ., Tho...",2002,False,article,random,train,0.000000
44816,"[`, :, :, the, term, ``, standard, model, ``, ...",2002,False,article,random,train,0.000000
49851,"[True, or, false, ,, the, situation, as, of, M...",2002,False,article,random,train,0.000000
89320,"[Next, ,, maybe, you, could, work, on, being, ...",2002,True,article,random,dev,0.444444
93890,"[This, page, will, need, disambiguation, .]",2002,True,article,random,train,0.000000
